In [1]:
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
# import mpl_toolkits.mplot3d as Axes3D

%matplotlib inline

# Datasets

Generate training, testing, and validation datasets

In [ ]:
#TODO
input_size = # ???
batch_size = 64
num_unrollings = 10

# LSTM RNN Definition

Define the long short-term memory (LSTM) recurrent neural network (RNN). Borrowing heavily from the Deep Learning Udacity course.

In [ ]:
num_nodes = 64  #TODO tune this!

graph = tf.Graph()
with graph.as_default():
    # Parameters
        # Combined input, forget, memory, and output weights for input and previous state. Also biases.
        ifcox = tf.Variable(tf.truncated_normal([input_size, 4*num_nodes], -0.1, 0.1))
        ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
        ifcob = tf.Variable(tf.zeros([1, 4*num_nodes]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([num_nodes, input_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([input_size]))
    
    # Definition of the cell computation
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        combined = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(combined[:, :num_nodes])
        forget_gate = tf.sigmoid(combined[:, num_nodes:2*num_nodes])
        update = combined[:, 2*num_nodes:3*num_nodes]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(combined[:, 3*num_nodes:])
        return output_gate * tf.tanh(state), state
    
    # Input data
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size, input_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.
    
    # Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    # Saving state across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,
                tf.concat(0, train_labels)))
    
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, input_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
            saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output,
            saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

# Run

Train the network on the dataset and observe the performance

In [2]:
num_steps = 1000
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized.')
    for step in num_steps:
        # Generate next training batch
        # Generate feed_dict
        # Run session
        if step % summary_frequency == 0:
            # Output some information about our training performance
        # Output some information about out validation performance

# Analyze

Analyze the performance and see what we can do with it

In [ ]:
#TODO